## About

This script uses stanford named entity recognizer to find the proper names in my women article corpus and exports a new csv column with the article text without proper nouns. At the end, it also tries to do a little analysis with those named entites.

## NER

In [11]:
import ner
import os
import re
import csv
from urllib import urlopen

In [12]:
import sys

csv.field_size_limit(sys.maxsize)
 

9223372036854775807

In [13]:
tagger = ner.SocketNER(host='localhost', port=8080)

In [14]:
# test
entities = tagger.get_entities("The dictatorship of President Nicolae Ceausescu caused extreme hardships \
                                for all but a few hundred thousand of Rumania's 23 million citizens. \
                                But in the case of mothers and babies, his rule apparently had the most \
                                tragic consequences. Mr. Ceausescu, who was ousted in a popular uprising a \
                                month ago, decreed in 1967, two years after he came to power, that Rumania's \
                                population, then about 22 million, should increase to 30 million. The reason he \
                                gave was simply that he wanted a bigger Rumania - an assertion widely interpreted \
                                now as an early indication of his megalomania. And to achieve his goal he banned \
                                abortions, made contraception illegal and ordered that Rumanian women of \
                                child-bearing age have five children each. No Precise Accounting Harsh fines were \
                                ordered for women caught having abortions, and doctors or medical technicians who \
                                a consultant for Murray Stopes International, a British charity that assi")
entities

{u'LOCATION': [u'Rumania', u'Rumania', u'Rumania'],
 u'ORGANIZATION': [u'Murray Stopes International'],
 u'PERSON': [u'Nicolae Ceausescu', u'Ceausescu']}

In [15]:
for key in entities:
    entities[key] = set(entities[key])
entities

{u'LOCATION': {u'Rumania'},
 u'ORGANIZATION': {u'Murray Stopes International'},
 u'PERSON': {u'Ceausescu', u'Nicolae Ceausescu'}}

In [16]:
#read csv and read into a list of dictionaries
women = []
with open('Data/Corpora/women-foreign.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        women.append(row)
women[1]

{'BYLINE': 'By DAVID BINDER, Special to The New York Times',
 'COUNTRY': 'ROMANIA\xc2\xa0(96%);',
 'COUNTRY_CODE': 'ROU',
 'COUNTRY_FINAL': 'Romania',
 'COUNTRY_MAJOR': 'ROMANIA',
 'COUNTRY_NR': 'ROMANIA\xc2\xa0(96%)',
 'COUNTRY_TOP_PERCENT': 'ROMANIA\xc2\xa0(96%)',
 'DATE': 'January 24, 1990',
 'LENGTH': '936 words',
 'PUBLICATION': 'NYT',
 'REGION': 'EECA',
 'SUBJECT': "ABORTION; BIRTH CONTROL AND FAMILY PLANNING; POPULATION; CHILDREN AND YOUTH; WOMEN; POLITICS AND GOVERNMENT \xc2\xa0PREGNANCY & CHILDBIRTH\xc2\xa0(90%);\xc2\xa0INFANTS & TODDLERS\xc2\xa0(90%);\xc2\xa0HEADS OF STATE & GOVERNMENT\xc2\xa0(90%);\xc2\xa0ABORTION\xc2\xa0(90%);\xc2\xa0WOMEN\xc2\xa0(90%); CHILDREN\xc2\xa0(89%);\xc2\xa0ORPHANS\xc2\xa0(89%);\xc2\xa0JAIL SENTENCING\xc2\xa0(88%);\xc2\xa0PARENTING\xc2\xa0(78%);\xc2\xa0WOMEN'S HEALTH\xc2\xa0(78%);\xc2\xa0POPULATION SIZE\xc2\xa0(78%);\xc2\xa0CONTRACEPTION\xc2\xa0(74%);\xc2\xa0LAW ENFORCEMENT\xc2\xa0(74%);",
 'TEXT': "The dictatorship of President Nicolae Ceausescu c

In [17]:
# remove named entities
for article in women:
    entities = tagger.get_entities(article['TEXT'])
    for key in entities:
        entities[key] = set(entities[key])
    article['entities'] = entities
    entities = [item for sublist in entities.values() for item in sublist]
    text = article['TEXT']
    for noun in entities:
        text = text.replace(noun.encode('utf-8'),'')
    article['TEXT-NO-NOUN'] = text

## Write File

In [18]:
keys = women[1].keys()
keys

['BYLINE',
 'TEXT-NO-NOUN',
 'PUBLICATION',
 'TITLE',
 'COUNTRY',
 'COUNTRY_FINAL',
 'YEAR',
 'UID',
 'COUNTRY_NR',
 'entities',
 'LENGTH',
 'COUNTRY_TOP_PERCENT',
 'COUNTRY_CODE',
 'TEXT',
 'DATE',
 'COUNTRY_MAJOR',
 'TYPE',
 'REGION',
 'SUBJECT']

In [19]:
with open('Data/Corpora/women-processed.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(women)